In [1]:
import torch
from torch.nn.utils import prune

from tqdm import tqdm

from transformers import AutoTokenizer, OPTForCausalLM, pipeline
from datasets import load_dataset

from utils.prune_utils import sparsegpt_prune

In [2]:
#DEVICE
device = 'cuda' if torch.cuda.is_available() else 'cpu'

#Load dataset
dataset = load_dataset('c4', "en", streaming=True)

In [3]:
# Calibrate model (get inputs to each layer with calibration data)

calibration_size=128
token_length=2048
calibration_batch_size=2

EPSILON = 0.01
B = 128
Bs = 128

# run model on batches of calibration data, then concatenate inputs
def split_model_calibration(model, tokenizer):
    batch_sentences = []
    for i, data in tqdm(enumerate(iter(dataset['train'])), total=calibration_size):
        if i < calibration_size + 1:
            if len(batch_sentences) >= calibration_batch_size:
                with torch.no_grad():
                    encoded_input = tokenizer(batch_sentences, return_tensors="pt",
                                              padding="max_length", max_length=token_length,
                                              truncation=True).to(device=device)
                    model(**encoded_input, labels=encoded_input.input_ids)
                    batch_sentences = []
            batch_sentences.append(data['text'])
        else:
            break

# Sparsify Model

In [4]:
from iterative_prune_finetune import iterative_sparsegpt_prune_tune
from utils.prehook_utils import put_input_hooks,remove_all_hooks
from utils.prune_utils import sparsegpt_prune
from utils.finetune_utils import finetune_model_inplace
from utils.save_utils import unmask_model
from utils.fsdp_finetune import fsdp_finetune
for model_size in ['opt-125m', 'opt-350m', 'opt-1.3b']:
    
    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(f'facebook/{model_size}', padding_side='left')
    SPARSITIES = [1, 0.9, 0.7, 0.5, 0.3, 0.2]#0.1, 0.2,0.3,0.5,0.7,0.9,1
    model = OPTForCausalLM.from_pretrained(f'facebook/{model_size}', 
                                           output_attentions=True, 
                                           output_hidden_states=True).to(device=device)
    model = torch.nn.DataParallel(model, device_ids=[0,1,2,3])
    for SPARSENESS in SPARSITIES:
        feature_hessians = {}
        #put_input_hooks(model=model, features=feature_hessians, storage_dir=storage_dir, offload_freq=10000, feature_storage_device='cpu')
        all_hooks = put_input_hooks(model=model, features=feature_hessians, feature_storage_device='cpu')
        split_model_calibration(model,tokenizer)
        for hook in all_hooks:
            hook.remove()
        sparsegpt_prune(model=model, model_name=model_size, 
                        feature_hessians=feature_hessians, 
                        EPSILON=EPSILON, SPARSENESS=SPARSENESS, B=B, Bs=Bs, save_model=False)
        del feature_hessians
        torch.cuda.empty_cache()
        #finetune_model_inplace(model=model, tokenizer=tokenizer, 
        #                       SPARSITY=SPARSENESS, device=device, EPOCH_COUNT=1, max_step=1000)
        #unmask_model(model=model)
        config = {"model": model, "lr": 2e-5, "num_epochs": 1,
                  "seed": 1, "batch_size": 16,
                  'model_name': model_size,
                  'sparsity': SPARSENESS,"train_steps": 10000,
                  'max_step': 10000, 'save_model': False}
        fsdp_finetune(config)
        pruned_model_name = f'{model_size}-finetuned-{SPARSENESS}'
        torch.save(model.state_dict(), f'pruned_models/{pruned_model_name}-iterative.pt')
    # Prune using the sparseGPT method, saves as pruned_models/{model_name}-{SPARSENESS}.pt WITHOUT mask
    #iterative_sparsegpt_prune_tune(model, model_size, SPARSITIES, feature_hessians, EPSILON, B, Bs, tokenizer, EPOCH_COUNT=10)

    del model
    torch.cuda.empty_cache()

  1%|          | 1/128 [00:00<00:17,  7.24it/s]/gs/gsfs0/users/asyed/.local/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
129it [01:11,  1.79it/s]                         
100%|██████████| 196/196 [00:17<00:00, 10.96it/s]


DistributedType.NO


  0%|          | 0/3 [00:00<?, ?it/s]/gs/gsfs0/users/asyed/.local/lib/python3.9/site-packages/torch/cuda/memory.py:282: FutureWarning: torch.cuda.reset_max_memory_allocated now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  warnings.warn(

 89%|████████▊ | 886/1000 [00:44<00:05, 20.58it/s]


 33%|███▎      | 1/3 [00:50<01:41, 50.51s/it]

Memory before entering the train : 478
Memory consumed at the end of the train (end-begin): 1756
Peak Memory consumed during the train (max-begin): 2789
Total Peak Memory consumed during the train (max): 3267


...

 67%|██████▋   | 2/3 [01:40<00:50, 50.27s/it]

Memory before entering the train : 2240
Memory consumed at the end of the train (end-begin): -1
Peak Memory consumed during the train (max-begin): 1027
Total Peak Memory consumed during the train (max): 3267



 90%|█████████ | 902/1000 [00:44<00:04, 20.51it/s]


100%|██████████| 3/3 [02:30<00:00, 50.25s/it]


Memory before entering the train : 2239
Memory consumed at the end of the train (end-begin): 1
Peak Memory consumed during the train (max-begin): 1029
Total Peak Memory consumed during the train (max): 3268
